In [ ]:
import csv
from time import sleep
from rfsoc_sam_custom.overlay import Overlay

overlay = Overlay(init_rf_clks=True)

In [ ]:
overlay.spectrum_analyzer().children[1]

In [ ]:
# Define Transmission Parameters
fc = int(500e6)  # 500 MHz Center Frequency
bw = int(100e3)  # 100 kHz Sweep Bandwidth
df = int(2.5e3)  # 2.5 kHz Sweep Granularity

# channel used
ACTIVE_CHANNEL = 0

# Configure Spectrum Analyser
overlay.radio.receiver.channels[ACTIVE_CHANNEL].frontend.config = {
    'centre_frequency': fc/1e6,
    'nyquist_stopband': 100,
    'decimation_factor': 2048,
    'fftsize': 65536,
    'post_process': 'none',
    'number_frames': 1,
    'window': 'rectangular',
    'spectrum_enable': True   
}


In [ ]:
CSV_FILE_NAME = 'analyser.csv'

# Create CSV file and write meta data
with open(CSV_FILE_NAME, 'w') as file:
    writer = csv.DictWriter(file, fieldnames=overlay.radio.receiver.channels[ACTIVE_CHANNEL].frontend.config.keys())
    
    # record meta data
    writer.writeheader()
    writer.writerow(overlay.radio.receiver.channels[ACTIVE_CHANNEL].frontend.config)
    
# write header for body of data
with open(CSV_FILE_NAME, 'a') as file:
    writer = csv.writer(file)
    
    writer.writerow([])  # write empty row for formatting
    # write x_data as well as a column header for target frequency
    x_data = overlay.radio.receiver.channels[ACTIVE_CHANNEL].frontend.analyser._spectrum_analyser.plot._x_data
    writer.writerow( ["Target Frequency"] + list(x_data) )

    # Turn on Transmitter
    overlay.radio.transmitter.channels[ACTIVE_CHANNEL].frontend.config = { 'centre_frequency': (fc-bw)/1e6, 'amplitude': 0.5, 'transmit_enable':True }
    
    # Start Sweep & Collect Data
    for freq in [(fc+d)/1e6 for d in range(-bw, bw+df, df)]:
        overlay.radio.transmitter.channels[ACTIVE_CHANNEL].frontend.config = { 'centre_frequency': freq, 'amplitude': 0.5 }
        sleep(1)  # Give some time for transmitter to re-configure
        
        y_data = overlay.radio.receiver.channels[ACTIVE_CHANNEL].frontend.analyser._spectrum_analyser.plot._y_data
        writer.writerow( [freq] + list(y_data) )
        
# Turn off transmission
overlay.radio.transmitter.channels[ACTIVE_CHANNEL].frontend.config = { 'centre_frequency': (fc-bw)/1e6, 'amplitude': 0.5, 'transmit_enable':False }